In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import pandas as pd 
import numpy as np

from pandas.core.reshape.reshape import get_dummies

print('Libraries imported')

Libraries imported


#Open data

In [3]:
data_proc = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/colabdata/train_data.csv", index_col=0)
# data_proc
# Open file in notepad, to eliminate error 0x83 utf
test_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/colabdata/TestDataAccomodation.csv", encoding="utf-8", index_col=0)
test_data = pd.DataFrame(test_data, columns = data_proc.columns[:-1]) 
test_data

,durationOfStay,gender,Age,kids,destinationCode
id,,,,,
1013319,2,F,38.0,0.0,BE
1009485,6,M,37.0,0.0,ES
1002725,1,M,37.0,1.0,ES
1001133,5,F,50.0,0.0,UK
1000794,4,F,42.0,1.0,DK
...,...,...,...,...,...
1007285,3,M,NaN,0.0,ES
1009715,4,F,35.0,NaN,NL
1012538,7,F,NaN,1.0,FI


#Analyze data
## Half of the data is categorial other half is continuous

In [ ]:
data_proc

,durationOfStay,gender,Age,kids,destinationCode,AcomType
Id,,,,,,
1010286,7,F,40.0,0.0,SE,Apt
1000152,5,M,NaN,0.0,NaN,Apt
1009019,4,F,39.0,1.0,ES,Hotel
1007840,5,F,45.0,0.0,NL,Hotel
1006939,5,M,45.0,1.0,UK,Hotel
...,...,...,...,...,...,...
1002141,2,F,42.0,0.0,ES,Apt
1009145,6,M,43.0,1.0,DK,Apt
1008594,4,M,39.0,0.0,IT,Hotel


#Fill in empty spaces
## Sample randomly

In [4]:
#Random Sampling missng values
data_proc = data_proc.apply(lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True), x))
#Random Sampling missng values
test_data = test_data.apply(lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True), x))

#Split data into train, test and validation sets
## prepare for cross validation to accommodate skewed data.

In [5]:
data = pd.get_dummies(data_proc.drop('AcomType', axis=1))
data["AcomType"] = data_proc["AcomType"]
# data
feature = data.drop('AcomType', axis=1)
target = data['AcomType']

X = feature.values
y = target.values

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, random_state=41)
X_train, X_valid, y_train, y_valid = train_test_split(X_trainval, y_trainval, random_state=3)

print("Training set{}\nValidation set{}\nTest set{}".format(X_train.shape, X_valid.shape, X_test.shape))

Training set(5550, 13)
Validation set(1851, 13)
Test set(2467, 13)


In [ ]:
data

,durationOfStay,Age,kids,gender_F,gender_M,destinationCode_BE,destinationCode_DK,destinationCode_ES,destinationCode_FI,destinationCode_IT,destinationCode_NL,destinationCode_SE,destinationCode_UK,AcomType
Id,,,,,,,,,,,,,,
1010286,7,40.0,0.0,1,0,0,0,0,0,0,0,1,0,Apt
1000152,5,39.0,0.0,0,1,0,0,0,0,0,0,1,0,Apt
1009019,4,39.0,1.0,1,0,0,0,1,0,0,0,0,0,Hotel
1007840,5,45.0,0.0,1,0,0,0,0,0,0,1,0,0,Hotel
1006939,5,45.0,1.0,0,1,0,0,0,0,0,0,0,1,Hotel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002141,2,42.0,0.0,1,0,0,0,1,0,0,0,0,0,Apt
1009145,6,43.0,1.0,0,1,0,1,0,0,0,0,0,0,Apt
1008594,4,39.0,0.0,0,1,0,0,0,0,1,0,0,0,Hotel


# Standardize data

In [6]:
scaler = MinMaxScaler()
scaler.fit(X_trainval)

X_trainval_scaled = scaler.transform(X_trainval)
X_test_scaled = scaler.transform(X_test)
X_train_scaled = scaler.transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

In [7]:
test_data = pd.get_dummies(test_data)

scaler = MinMaxScaler()
scaler.fit(test_data)

test_data_scaled = scaler.transform(test_data)
# test_data_scaled

# Results

In [ ]:
# ranfor = list(rf.predict(test_data_scaled))
# logger = list(logreg.predict(test_data_scaled))
# svcer = list(svc.predict(test_data_scaled))

# res_df = pd.DataFrame({'RF': ranfor, 'LogReg': logger, 'SVC': svcer})
# res_df[res_df.RF == res_df.SVC]

In [ ]:
# def get_results(best_score, best_parameters, train_score, test_score):
#   print("Best score validation set {:.2f}: ".format(best_score))
#   print("Best parameters: ", best_parameters)
#   print("Train set with best parameters {:.2f}: ".format(train_score))
#   print("Test set with best parameters {:.2f}: ".format(test_score))
#   return

#Support Vector Machine

In [12]:
def support_vector_machine(X_train_scaled,  y_train, X_valid_scaled, y_valid, X_trainval_scaled,  y_trainval, X_test_scaled, y_test):
    best_score = 0

    for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]:
      for gamma in ["auto", "scale"]:
        for kernel in ["linear", "poly", "rbf", "sigmoid"]:
          svc = SVC(kernel=kernel, C=C, gamma=gamma)
          svc.fit(X_train_scaled,  y_train)
          score = svc.score(X_valid_scaled, y_valid)

          if score > best_score:
            best_score = score
            best_parameters = {"C":C,
                              'gamma': gamma,
                              'kernel': kernel}

    svc = SVC(**best_parameters)
    clf = svc.fit(X_trainval_scaled,  y_trainval)
    train_score = svc.score(X_trainval_scaled,  y_trainval)
    test_score = svc.score(X_test_scaled, y_test)

    return clf, cleanup_results(best_score, best_parameters, train_score, test_score)

clf, results = support_vector_machine(X_train_scaled,  y_train, X_valid_scaled, y_valid, X_trainval_scaled,  y_trainval, X_test_scaled, y_test)

clf, results

TypeError: ignored

In [15]:
def cleanup_results(best_score, best_parameters, train_score, test_score):
  return {'accuracy': ["{:.2f}".format(best_score)], 
          'best_parameters': best_parameters,
          'train_score': ["{:.2f}".format(train_score)], 
          'test_score': ["{:.2f}".format(test_score)]}

In [ ]:
# Imports
from sklearn.datasets import make_blobs

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix

# Configuration options
blobs_random_seed = 42
centers = [(0,0), (5,5)]
cluster_std = 1.5
frac_test_split = 0.33
num_features_for_samples = 2
num_samples_total = 1000

# Generate data
inputs, targets = make_blobs(n_samples = num_samples_total, centers = centers, n_features = num_features_for_samples, cluster_std = cluster_std)
X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=frac_test_split, random_state=blobs_random_seed)

# Save and load temporarily
np.save('./datasv.npy', (X_train, X_test, y_train, y_test))
X_train, X_test, y_train, y_test = np.load('./datasv.npy', allow_pickle=True)

# Generate scatter plot for training data 
plt.scatter(X_train[:,0], X_train[:,1])
plt.title('Linearly separable data')
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

# Initialize SVM classifier
clf = svm.SVC(kernel='linear')

# Fit data
clf = clf.fit(X_train, y_train) 

In [ ]:
# Get support vectors themselves
support_vectors = clf.support_vectors_

# Visualize support vectors
plt.scatter(X_train[:,0], X_train[:,1])
plt.scatter(support_vectors[:,0], support_vectors[:,1], color='red')
plt.title('Linearly separable data with support vectors')
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

#Logistic regression

In [ ]:
def logistic_regression(X_train_scaled,  y_train, X_valid_scaled, y_valid, X_trainval_scaled,  y_trainval, X_test_scaled, y_test):
    best_score = 0

    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
      for solver in ["newton-cg", "sag", "saga", "lbfgs"]:
          logreg = LogisticRegression(max_iter=5000, multi_class='multinomial', C=C,
                                      solver=solver, penalty='l2')
          logreg.fit(X_train_scaled,  y_train)
          score = logreg.score(X_valid_scaled, y_valid)

          if score > best_score:
            best_score = score
            best_parameters = {"C":C,
                              'solver': solver}

    logreg = LogisticRegression(**best_parameters)
    clf = logreg.fit(X_trainval_scaled,  y_trainval)
    train_score = logreg.score(X_trainval_scaled,  y_trainval)
    test_score = logreg.score(X_test_scaled, y_test)

    return clf, best_score, best_parameters, train_score, test_score

logistic_regression(X_train_scaled,  y_train, X_valid_scaled, y_valid, X_trainval_scaled,  y_trainval, X_test_scaled, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


(0.5575364667747164,
 {'C': 0.001, 'solver': 'newton-cg'},
 0.5665450614781786,
 0.5670855289825699)

#RandomForestClassifier

In [16]:
def random_forest_classifier(X_train_scaled,  y_train, X_valid_scaled, y_valid, X_trainval_scaled,  y_trainval, X_test_scaled, y_test):
    best_score = 0

    for criterion in ['gini', 'entropy']:
      for max_depth in range(1,11):
        for max_features in ["auto", "sqrt", "log2"]:
          rf = RandomForestClassifier(criterion=criterion, max_features=max_features, max_depth=max_depth)
          rf.fit(X_train_scaled,  y_train)
          score = rf.score(X_valid_scaled, y_valid)

          if score > best_score:
            best_score = score
            best_parameters = {"max_features":max_features,
                              'max_depth': max_depth,
                              'criterion': criterion}

    rf = RandomForestClassifier(**best_parameters)
    clf = rf.fit(X_trainval_scaled,  y_trainval)
    train_score = rf.score(X_trainval_scaled,  y_trainval)
    test_score = rf.score(X_test_scaled, y_test)

    return cleanup_results(best_score, best_parameters, train_score, test_score)

results = random_forest_classifier(X_train_scaled,  y_train, X_valid_scaled, y_valid, X_trainval_scaled,  y_trainval, X_test_scaled, y_test)
results

{'accuracy': ['0.76'],
 'best_parameters': {'criterion': 'gini',
  'max_depth': 10,
  'max_features': 'log2'},
 'test_score': ['0.77'],
 'train_score': ['0.81']}

In [ ]:
test ={
 'Age': [40.0] ,
 'destinationCode': ['SE'],
 'durationOfStay': [7] ,
 'gender': ['F'] ,
 'kids': [0.0] }

test = get_dummies(pd.DataFrame(test), ['destinationCode', 'gender'] )

empty_df = get_dummies(data_proc.drop("AcomType", axis=1)).reset_index().drop([x for x in range(len(data_proc))], axis=0)

new_test = pd.concat([empty_df, test], sort=False).fillna(0).drop("Id", axis=1)

test = list(new_test.iloc[0])
test

[7.0, 40.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

In [ ]:
def get_recommendation(clf, profiles):
    np_profiles = np.array(profiles) 
    print(np_profiles)

    test ={
    'durationOfStay': np_profiles[:, 0],
    'gender': np_profiles[:, 1], 
    'Age': np_profiles[:, 2],
    'kids': np_profiles[:, 3], 
    'destinationCode': np_profiles[:, 4]
    }

    test = get_dummies(pd.DataFrame(test), columns =['destinationCode', 'gender'] )

    empty_df = get_dummies(data_proc.drop("AcomType", axis=1)).reset_index().drop([x for x in range(len(data_proc))], axis=0)

    new_test = pd.concat([empty_df, test], sort=False).fillna(0).drop("Id", axis=1)

    test = list(new_test.values)
        
    return clf.predict(test)

results = get_recommendation(clf, [[2, 'F', 38.0, 0.0, 'BE'], 
 [6, 'M', 37.0, 0.0, 'ES'], 
 [1, 'M', 37.0, 1.0, 'ES']])

results 


#results = get_recommendation(clf, test_data.values  ) #[[40, 'SE', 7, 'F', 0], [22, 'SE', 7, 'M', 0] ] )

[['2' 'F' '38.0' '0.0' 'BE']
 ['6' 'M' '37.0' '0.0' 'ES']
 ['1' 'M' '37.0' '1.0' 'ES']]


array(['Apt', 'Apt', 'Apt'], dtype=object)

In [ ]:
#test_data.values

# https://medium.com/nmc-techblog/easy-drag-and-drop-in-react-22778b30ba37
# https://medium.com/analytics-vidhya/classification-method-for-estimating-the-numbers-of-rings-of-abalone-bb13264dd186


In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler

In [ ]:
# pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
# pipe.fit(X_train, y_train)
# pipe.score(X_test, y_test)